# Nettoyage ou modification du dataset

Passage de la version 1 à la version 2 (dynamique) 
Modification au besoin, ajout / suppression / modification des colonnes

In [46]:
import pandas as pd
from datetime import *
import numpy as np

def classCoffee(points):
    if points >= 90:
        # Outstanding
        return 1
    if points <90 and points >= 85:
        # Excellent
        return 2
    if points <85 and points >= 80:
        # Very Good
        return 3 
    else:
        return 4

# Read the CSV file (utf-8 encoded)
df = pd.read_csv('./data/DataRisaralda_v1_utf-8.csv', header=0, sep=',', index_col=0, parse_dates=False, encoding='utf-8')
print(df.shape)


df
df.rename(columns={'EPSG:3116_X': 'EPSG:4326_X', 'EPSG:3116_Y': 'EPSG:4326_Y'}, inplace=True)
df.rename(columns={'CENT_X': 'EPSG:3116_X', 'CENT_Y': 'EPSG:3116_Y'}, inplace=True)

(975, 139)


In [47]:
try:
    df.drop('Cedula', axis=1, inplace=True)
except:
    print("No such column")
    

try:
    df.drop('Fecha Recpt', axis=1, inplace=True)
except:
    print("No such column")
    
    
try:
    df.drop('Factor de Rendimiento', axis=1, inplace=True)
except:
    print("No such column")    
    

In [48]:
cols = ['Malla 17-18','Malla 14-16','Malla <13','Negros y vinagres','Broca','Broca de punto','Veteado','Mordido',
         'Inmaduro','Flojo','Sobresecado','Arrugado','Aplastado','Cristalizado','Reposado','Granizo','Conchas',
         'Partido','Ambar','Defectos totales','Humedad (%)','Merma (%)','Aroma-Fragancia',
         'Acidez','Cuerpo','Sabor',
         'Sabor residual','Dulzor','Uniformidad','Balance','Taza limpia','Puntaje catador','Puntaje Total',
         'prec1','prec2','prec3','prec4','prec5','prec6','prec7','prec8','prec9','prec10','tmin1','tmin2','tmin3','tmin4','tmin5',
         'tmin6','tmin7','tmin8','tmin9','tmin10','tmax1','tmax2','tmax3','tmax4','tmax5','tmax6','tmax7','tmax8','tmax9','tmax10',
         'tmean1','tmean2','tmean3','tmean4','tmean5','tmean6','tmean7','tmean8','tmean9','tmean10','dtr1','dtr2','dtr3','dtr4',
         'dtr5','dtr6','dtr7','dtr8','dtr9','dtr10']
defectos = ['Negros y vinagres','Broca','Broca de punto','Veteado','Mordido',
         'Inmaduro','Flojo','Sobresecado','Arrugado','Aplastado','Cristalizado','Reposado','Granizo','Conchas',
         'Partido','Ambar']

for index, row in df.iterrows():
    date = datetime.strptime(str(row['Fecha Analysis']), '%Y-%m-%d %H:%M:%S')
    df.set_value(index,'Fecha Analysis', date)

# Ajout des moyennes sur les 10 mois de mesures

In [49]:
precs = ['prec1','prec2','prec3','prec4','prec5','prec6','prec7','prec8','prec9','prec10']
tmins = ['tmin1','tmin2','tmin3','tmin4','tmin5','tmin6','tmin7','tmin8','tmin9','tmin10']
tmaxs = ['tmax1','tmax2','tmax3','tmax4','tmax5','tmax6','tmax7','tmax8','tmax9','tmax10']
tmeans = ['tmean1','tmean2','tmean3','tmean4','tmean5','tmean6','tmean7','tmean8','tmean9','tmean10']
dtrs = ['dtr1','dtr2','dtr3','dtr4','dtr5','dtr6','dtr7','dtr8','dtr9','dtr10']

# Moyenne sur les 10 mois de mesures

df['PrecTotalAvg'] = 0
df['TminTotalAvg'] = 0
df['TmaxTotalAvg'] = 0
df['TmeanTotalAvg'] = 0
df['DtrTotalAvg'] = 0

df['PrecTotal'] = 0
df['TminTotal'] = 0
df['TmaxTotal'] = 0
df['TmeanTotal'] = 0
df['DtrTotal'] = 0


for index, row in df.iterrows():
    
    df.set_value(index,'PrecTotalAvg', row[precs].sum() / len(precs))
    df.set_value(index,'TminTotalAvg', row[tmins].sum() / len(precs))
    df.set_value(index,'TmaxTotalAvg', row[tmaxs].sum() / len(precs))
    df.set_value(index,'TmeanTotalAvg', row[tmeans].sum() / len(precs))
    df.set_value(index,'DtrTotalAvg', row[dtrs].sum() / len(precs))
    
    df.set_value(index,'PrecTotal', row[precs].sum())
    df.set_value(index,'TminTotal', row[tmins].sum())
    df.set_value(index,'TmaxTotal', row[tmaxs].sum())
    df.set_value(index,'TmeanTotal', row[tmeans].sum())
    df.set_value(index,'DtrTotal', row[dtrs].sum())


# Conversion de la luminosité en valeurs numériques


In [50]:
df.LUMINOSID.unique()

array([u'Sol', u'Semi Sombra', u'Sombra'], dtype=object)

In [51]:
def convert(lum):
    if lum == "Sol":
        return 2
    elif lum == "Semi Sombra":
        return 1
    elif lum == "Sombra":
        return 0
    else:
        return 0
    
for index, row in df.iterrows():
    df.set_value(index, 'LUMINOSID', convert(row['LUMINOSID']))
    

# Correction semi-manuelle des erreurs sur les points

In [52]:
catacionpts = ['Aroma-Fragancia','Acidez','Cuerpo','Sabor residual',
               'Sabor','Dulzor','Uniformidad','Balance','Taza limpia','Puntaje catador']

for index, row in df.iterrows():
        if row['Puntaje Total'] > 100:
            print index, row['Puntaje Total']
            df.set_value(index, 'Puntaje catador', 6.9)
            print row[catacionpts].sum()
            
            df.set_value(index, 'Puntaje Total', row[catacionpts].sum())

for index, row in df.iterrows():
        if row['Puntaje Total'] > 100:
            print index, row['Puntaje Total']

for index, row in df.iterrows():
        if np.isnan(row['Acidez']) or np.isnan(row['Puntaje Total']):
            df.set_value(index, catacionpts, 0)
            df.set_value(index, 'Puntaje Total', 0)
        if np.isnan(row['Defectos totales']):
            df.set_value(index, 'Defectos totales', row[defectos].sum())
            


# Convertion SICA

In [53]:
print df['SICA 1'].dtype

df['SICA 1'] = df['SICA 1'].astype(np.int64)
df['SICA 1'] = df['SICA 1'].astype(str)
print df['SICA 1'].dtype



float64
object


In [54]:
df

,Municipio,Vereda,Finca,SICA,CEDULA 1,SICA 1,REPRESENTA,VARIEDAD,EPSG:3116_X,EPSG:3116_Y,...,franco_L3,arcilloso_L3,limoso_L3,arenoso_L3,cascajoso_L3,PrecTotal,TminTotal,TmaxTotal,TmeanTotal,DtrTotal
0,Balboa,La Cienaga,Palestina,6.607501e+09,14257374,6607500996,Puntual,COLOMBIA,790897.6106,1037547.461,...,1.0,1.0,1.0,0.0,0.0,2136,167,259,213,91
1,Balboa,La Cienaga,La Albania,6.607501e+09,2280361,6607500981,Puntual,COLOMBIA,790598.7547,1037800.494,...,1.0,1.0,1.0,0.0,0.0,2108,164,245,204,81
2,Balboa,El Chuzcal,La Esperanza,6.607501e+09,4379287,6607501034,Puntual,COLOMBIA,789390.5633,1038560.999,...,1.0,1.0,1.0,0.0,0.0,2017,176,275,225,99
3,Balboa,Llanogrande,Altamira,6.607500e+09,4378752,6607500081,Puntual,T?PICA,792672.5954,1042507.264,...,1.0,0.0,1.0,0.0,0.0,1997,182,285,233,103
4,Balboa,Cocohondo,La Palma,6.607500e+09,65807427,6607500108,Puntual,COLOMBIA,792341.9870,1041185.481,...,1.0,1.0,1.0,0.0,0.0,1884,186,292,239,105
5,Balboa,La Floresta,La Paiva,6.607500e+09,42138525,6607500309,Alinderado,COLOMBIA,789867.6532,1036981.771,...,1.0,0.0,1.0,0.0,0.0,2127,165,251,208,86
6,Balboa,Llanogrande,El Llano,6.607500e+09,4378773,6607500014,Puntual,COLOMBIA,791308.3376,1041652.777,...,1.0,0.0,1.0,0.0,0.0,2163,167,255,211,88
7,Balboa,Carminales,El Guadualito,6.607500e+09,30412314,6607500148,Puntual,COLOMBIA,794450.1679,1039293.955,...,1.0,1.0,1.0,0.0,0.0,2034,178,280,229,101
8,Balboa,Tambores,El Bosque,6.607501e+09,4551819,6607500516,Puntual,COLOMBIA,789741.6682,1033348.814,...,1.0,0.0,1.0,0.0,0.0,2072,168,263,216,94
9,Balboa,La Floresta,La Mejora,6.607501e+09,24512358,6607501187,Puntual,COLOMBIA,790532.0700,1036504.194,...,1.0,1.0,1.0,0.0,0.0,2131,166,256,211,89


In [55]:
print type(df.isnull().any())

for a, e in df.isnull().any().iteritems():
    print a, "\t\t", e

<class 'pandas.core.series.Series'>
Municipio 		True
Vereda 		True
Finca 		True
SICA 		True
CEDULA 1 		False
SICA 1 		False
REPRESENTA 		False
VARIEDAD 		False
EPSG:3116_X 		False
EPSG:3116_Y 		False
EPSG:4326_Y 		False
EPSG:4326_X 		False
ASNM 		False
LUMINOSID 		False
Fecha Analysis 		False
Malla 17-18 		True
Malla 14-16 		True
Malla <13 		True
Negros y vinagres 		False
Broca 		False
Broca de punto 		False
Veteado 		False
Mordido 		True
Inmaduro 		False
Flojo 		False
Sobresecado 		False
Arrugado 		False
Aplastado 		False
Cristalizado 		False
Reposado 		False
Granizo 		False
Conchas 		True
Partido 		False
Ambar 		True
Defectos totales 		False
Quaker (Nb Granos) 		True
Fluorescentes (Nb Granos) 		True
Pergamino 		True
Almendra 		True
AlmendraTotal 		True
AlmendraSana 		True
UV 		True
Olor 		True
Humedad (%) 		False
Merma (%) 		False
Aroma-Fragancia 		False
Acidez 		False
Cuerpo 		False
Sabor 		False
Sabor residual 		False
Dulzor 		False
Uniformidad 		False
Balance 		False
Taza limpia 	

# Suppression des doublons

Basé sur la date de test et le numéro SICA

In [56]:
df = df.drop_duplicates(subset=['Fecha Analysis', 'SICA 1'], keep='first', inplace=False)
df = df.reset_index(drop=True)

In [57]:
df
#df[cols] = df[cols].apply(pd.to_numeric)

,Municipio,Vereda,Finca,SICA,CEDULA 1,SICA 1,REPRESENTA,VARIEDAD,EPSG:3116_X,EPSG:3116_Y,...,franco_L3,arcilloso_L3,limoso_L3,arenoso_L3,cascajoso_L3,PrecTotal,TminTotal,TmaxTotal,TmeanTotal,DtrTotal
0,Balboa,La Cienaga,Palestina,6.607501e+09,14257374,6607500996,Puntual,COLOMBIA,790897.6106,1037547.461,...,1.0,1.0,1.0,0.0,0.0,2136,167,259,213,91
1,Balboa,La Cienaga,La Albania,6.607501e+09,2280361,6607500981,Puntual,COLOMBIA,790598.7547,1037800.494,...,1.0,1.0,1.0,0.0,0.0,2108,164,245,204,81
2,Balboa,El Chuzcal,La Esperanza,6.607501e+09,4379287,6607501034,Puntual,COLOMBIA,789390.5633,1038560.999,...,1.0,1.0,1.0,0.0,0.0,2017,176,275,225,99
3,Balboa,Llanogrande,Altamira,6.607500e+09,4378752,6607500081,Puntual,T?PICA,792672.5954,1042507.264,...,1.0,0.0,1.0,0.0,0.0,1997,182,285,233,103
4,Balboa,Cocohondo,La Palma,6.607500e+09,65807427,6607500108,Puntual,COLOMBIA,792341.9870,1041185.481,...,1.0,1.0,1.0,0.0,0.0,1884,186,292,239,105
5,Balboa,La Floresta,La Paiva,6.607500e+09,42138525,6607500309,Alinderado,COLOMBIA,789867.6532,1036981.771,...,1.0,0.0,1.0,0.0,0.0,2127,165,251,208,86
6,Balboa,Llanogrande,El Llano,6.607500e+09,4378773,6607500014,Puntual,COLOMBIA,791308.3376,1041652.777,...,1.0,0.0,1.0,0.0,0.0,2163,167,255,211,88
7,Balboa,Carminales,El Guadualito,6.607500e+09,30412314,6607500148,Puntual,COLOMBIA,794450.1679,1039293.955,...,1.0,1.0,1.0,0.0,0.0,2034,178,280,229,101
8,Balboa,Tambores,El Bosque,6.607501e+09,4551819,6607500516,Puntual,COLOMBIA,789741.6682,1033348.814,...,1.0,0.0,1.0,0.0,0.0,2072,168,263,216,94
9,Balboa,La Floresta,La Mejora,6.607501e+09,24512358,6607501187,Puntual,COLOMBIA,790532.0700,1036504.194,...,1.0,1.0,1.0,0.0,0.0,2131,166,256,211,89


In [58]:
print(df.shape)

(767, 141)


# Suppression des dégustations avec même SICA à moins de 3 mois d'écart

    Voir: analyse de l'occurence des cafés dans 2_CoffeDataAnalysis


In [61]:

# Certains cafés on été testés plusieurs fois, parfois même 2 fois dans la même année mais à plusieurs mois d'intervale. 

dOccurences = df['SICA 1'].value_counts().to_dict() #.to_frame(name='Occurence')

dfOccurences = pd.DataFrame.from_dict(dOccurences, orient='index')

# Ajout d'une colonne contenant le nombre d'occurence

df['occurence'] = 0

# Fonctionne pas dans datacleaning...?
df['SICA 1'] = df['SICA 1'].astype(str)




Municipio
Vereda
Finca
SICA
CEDULA 1
SICA 1
REPRESENTA
VARIEDAD
EPSG:3116_X
EPSG:3116_Y
EPSG:4326_Y
EPSG:4326_X
ASNM
LUMINOSID
Fecha Analysis
Malla 17-18
Malla 14-16
Malla <13
Negros y vinagres
Broca
Broca de punto
Veteado
Mordido
Inmaduro
Flojo
Sobresecado
Arrugado
Aplastado
Cristalizado
Reposado
Granizo
Conchas
Partido
Ambar
Defectos totales
Quaker (Nb Granos)
Fluorescentes (Nb Granos)
Pergamino
Almendra
AlmendraTotal
AlmendraSana
UV
Olor
Humedad (%)
Merma (%)
Aroma-Fragancia
Acidez
Cuerpo
Sabor
Sabor residual
Dulzor
Uniformidad
Balance
Taza limpia
Puntaje catador
Puntaje Total
Taza 1
Taza 2
Taza 3
Taza 4
Taza 5
prec1
prec2
prec3
prec4
prec5
prec6
prec7
prec8
prec9
prec10
tmin1
tmin2
tmin3
tmin4
tmin5
tmin6
tmin7
tmin8
tmin9
tmin10
tmax1
tmax2
tmax3
tmax4
tmax5
tmax6
tmax7
tmax8
tmax9
tmax10
tmean1
tmean2
tmean3
tmean4
tmean5
tmean6
tmean7
tmean8
tmean9
tmean10
dtr1
dtr2
dtr3
dtr4
dtr5
dtr6
dtr7
dtr8
dtr9
dtr10
year
PrecTotalAvg
TminTotalAvg
TmaxTotalAvg
TmeanTotalAvg
DtrTotalAvg
Soi

In [63]:
for index, row in df.iterrows():
    df.set_value(index, 'occurence', dOccurences.get(str(row['SICA 1'])))

#print df['occurence']

print df['occurence'].value_counts()



df


1    631
2    118
3     18
Name: occurence, dtype: int64


,Municipio,Vereda,Finca,SICA,CEDULA 1,SICA 1,REPRESENTA,VARIEDAD,EPSG:3116_X,EPSG:3116_Y,...,arcilloso_L3,limoso_L3,arenoso_L3,cascajoso_L3,PrecTotal,TminTotal,TmaxTotal,TmeanTotal,DtrTotal,occurence
0,Balboa,La Cienaga,Palestina,6.607501e+09,14257374,6607500996,Puntual,COLOMBIA,790897.6106,1037547.461,...,1.0,1.0,0.0,0.0,2136,167,259,213,91,1
1,Balboa,La Cienaga,La Albania,6.607501e+09,2280361,6607500981,Puntual,COLOMBIA,790598.7547,1037800.494,...,1.0,1.0,0.0,0.0,2108,164,245,204,81,1
2,Balboa,El Chuzcal,La Esperanza,6.607501e+09,4379287,6607501034,Puntual,COLOMBIA,789390.5633,1038560.999,...,1.0,1.0,0.0,0.0,2017,176,275,225,99,1
3,Balboa,Llanogrande,Altamira,6.607500e+09,4378752,6607500081,Puntual,T?PICA,792672.5954,1042507.264,...,0.0,1.0,0.0,0.0,1997,182,285,233,103,1
4,Balboa,Cocohondo,La Palma,6.607500e+09,65807427,6607500108,Puntual,COLOMBIA,792341.9870,1041185.481,...,1.0,1.0,0.0,0.0,1884,186,292,239,105,1
5,Balboa,La Floresta,La Paiva,6.607500e+09,42138525,6607500309,Alinderado,COLOMBIA,789867.6532,1036981.771,...,0.0,1.0,0.0,0.0,2127,165,251,208,86,1
6,Balboa,Llanogrande,El Llano,6.607500e+09,4378773,6607500014,Puntual,COLOMBIA,791308.3376,1041652.777,...,0.0,1.0,0.0,0.0,2163,167,255,211,88,1
7,Balboa,Carminales,El Guadualito,6.607500e+09,30412314,6607500148,Puntual,COLOMBIA,794450.1679,1039293.955,...,1.0,1.0,0.0,0.0,2034,178,280,229,101,1
8,Balboa,Tambores,El Bosque,6.607501e+09,4551819,6607500516,Puntual,COLOMBIA,789741.6682,1033348.814,...,0.0,1.0,0.0,0.0,2072,168,263,216,94,1
9,Balboa,La Floresta,La Mejora,6.607501e+09,24512358,6607501187,Puntual,COLOMBIA,790532.0700,1036504.194,...,1.0,1.0,0.0,0.0,2131,166,256,211,89,1


In [64]:
df_1 = df[df['occurence'] == 2]

fields = ['SICA 1','Fecha Analysis','year']




df_1 = df_1[fields].sort_values(['SICA 1'])
df

,Municipio,Vereda,Finca,SICA,CEDULA 1,SICA 1,REPRESENTA,VARIEDAD,EPSG:3116_X,EPSG:3116_Y,...,arcilloso_L3,limoso_L3,arenoso_L3,cascajoso_L3,PrecTotal,TminTotal,TmaxTotal,TmeanTotal,DtrTotal,occurence
0,Balboa,La Cienaga,Palestina,6.607501e+09,14257374,6607500996,Puntual,COLOMBIA,790897.6106,1037547.461,...,1.0,1.0,0.0,0.0,2136,167,259,213,91,1
1,Balboa,La Cienaga,La Albania,6.607501e+09,2280361,6607500981,Puntual,COLOMBIA,790598.7547,1037800.494,...,1.0,1.0,0.0,0.0,2108,164,245,204,81,1
2,Balboa,El Chuzcal,La Esperanza,6.607501e+09,4379287,6607501034,Puntual,COLOMBIA,789390.5633,1038560.999,...,1.0,1.0,0.0,0.0,2017,176,275,225,99,1
3,Balboa,Llanogrande,Altamira,6.607500e+09,4378752,6607500081,Puntual,T?PICA,792672.5954,1042507.264,...,0.0,1.0,0.0,0.0,1997,182,285,233,103,1
4,Balboa,Cocohondo,La Palma,6.607500e+09,65807427,6607500108,Puntual,COLOMBIA,792341.9870,1041185.481,...,1.0,1.0,0.0,0.0,1884,186,292,239,105,1
5,Balboa,La Floresta,La Paiva,6.607500e+09,42138525,6607500309,Alinderado,COLOMBIA,789867.6532,1036981.771,...,0.0,1.0,0.0,0.0,2127,165,251,208,86,1
6,Balboa,Llanogrande,El Llano,6.607500e+09,4378773,6607500014,Puntual,COLOMBIA,791308.3376,1041652.777,...,0.0,1.0,0.0,0.0,2163,167,255,211,88,1
7,Balboa,Carminales,El Guadualito,6.607500e+09,30412314,6607500148,Puntual,COLOMBIA,794450.1679,1039293.955,...,1.0,1.0,0.0,0.0,2034,178,280,229,101,1
8,Balboa,Tambores,El Bosque,6.607501e+09,4551819,6607500516,Puntual,COLOMBIA,789741.6682,1033348.814,...,0.0,1.0,0.0,0.0,2072,168,263,216,94,1
9,Balboa,La Floresta,La Mejora,6.607501e+09,24512358,6607501187,Puntual,COLOMBIA,790532.0700,1036504.194,...,1.0,1.0,0.0,0.0,2131,166,256,211,89,1


In [65]:
from datetime import datetime

def days_between(d1, d2):
    d1 = datetime.strptime(d1, "%Y-%m-%d %H:%M:%S")
    d2 = datetime.strptime(d2, "%Y-%m-%d %H:%M:%S")
    return abs((d2 - d1).days)



In [66]:
item = 1
oldDate = 1
todel = []

for index, row in df_1.iterrows():
    
    if item == 1:
        item = 2
        firstDate = row['Fecha Analysis']
        firstsica = row['SICA 1']
        continue
    if item == 2:
        secondDate = row['Fecha Analysis']
        secondsica = row['SICA 1']
        space = days_between(str(firstDate), str(secondDate))
        if space < 90:
            todel.append(index)
            print("Sica1 {} - Sica2 {}".format(firstsica,secondsica))
            print("Date1 {} - Date2 {}".format(firstDate,secondDate))
        item = 1

print todel

Sica1 6604502461 - Sica2 6604502461
Date1 2016-07-22 00:00:00 - Date2 2016-07-17 00:00:00
Sica1 6608800780 - Sica2 6608800780
Date1 2016-07-22 00:00:00 - Date2 2016-05-24 00:00:00
Sica1 6608800799 - Sica2 6608800799
Date1 2016-07-25 00:00:00 - Date2 2016-09-14 00:00:00
Sica1 6608800899 - Sica2 6608800899
Date1 2016-07-22 00:00:00 - Date2 2016-07-24 00:00:00
Sica1 6608800959 - Sica2 6608800959
Date1 2016-07-24 00:00:00 - Date2 2016-07-21 00:00:00
Sica1 6617000764 - Sica2 6617000764
Date1 2016-05-16 00:00:00 - Date2 2016-07-22 00:00:00
Sica1 6638300445 - Sica2 6638300445
Date1 2016-06-23 00:00:00 - Date2 2016-06-02 00:00:00
Sica1 6644000388 - Sica2 6644000388
Date1 2016-07-22 00:00:00 - Date2 2016-06-23 00:00:00
Sica1 6644000596 - Sica2 6644000596
Date1 2016-07-22 00:00:00 - Date2 2016-05-21 00:00:00
Sica1 6644001113 - Sica2 6644001113
Date1 2016-07-22 00:00:00 - Date2 2016-05-16 00:00:00
Sica1 6645600841 - Sica2 6645600841
Date1 2011-12-07 00:00:00 - Date2 2011-12-29 00:00:00
Sica1 6659

In [67]:
print(len(todel))
print(len(df_1.index))

df = df.drop(todel)
todel = []


21
118


In [69]:
df = df.reset_index(drop=True)

In [70]:
print df.shape

(746, 142)


# Ecriture du dataset V2 

Contient toutes les données (nom de la ferme (Finca) etc)

In [71]:
df.to_csv("./data/DataRisaralda_v2_utf-8.csv", sep=',', encoding='utf-8')
df.to_csv("./data/DataRisaralda_v2_iso-8859-1.csv", sep=',', encoding='iso-8859-1')

In [72]:
df

,Municipio,Vereda,Finca,SICA,CEDULA 1,SICA 1,REPRESENTA,VARIEDAD,EPSG:3116_X,EPSG:3116_Y,...,arcilloso_L3,limoso_L3,arenoso_L3,cascajoso_L3,PrecTotal,TminTotal,TmaxTotal,TmeanTotal,DtrTotal,occurence
0,Balboa,La Cienaga,Palestina,6.607501e+09,14257374,6607500996,Puntual,COLOMBIA,790897.6106,1037547.461,...,1.0,1.0,0.0,0.0,2136,167,259,213,91,1
1,Balboa,La Cienaga,La Albania,6.607501e+09,2280361,6607500981,Puntual,COLOMBIA,790598.7547,1037800.494,...,1.0,1.0,0.0,0.0,2108,164,245,204,81,1
2,Balboa,El Chuzcal,La Esperanza,6.607501e+09,4379287,6607501034,Puntual,COLOMBIA,789390.5633,1038560.999,...,1.0,1.0,0.0,0.0,2017,176,275,225,99,1
3,Balboa,Llanogrande,Altamira,6.607500e+09,4378752,6607500081,Puntual,T?PICA,792672.5954,1042507.264,...,0.0,1.0,0.0,0.0,1997,182,285,233,103,1
4,Balboa,Cocohondo,La Palma,6.607500e+09,65807427,6607500108,Puntual,COLOMBIA,792341.9870,1041185.481,...,1.0,1.0,0.0,0.0,1884,186,292,239,105,1
5,Balboa,La Floresta,La Paiva,6.607500e+09,42138525,6607500309,Alinderado,COLOMBIA,789867.6532,1036981.771,...,0.0,1.0,0.0,0.0,2127,165,251,208,86,1
6,Balboa,Llanogrande,El Llano,6.607500e+09,4378773,6607500014,Puntual,COLOMBIA,791308.3376,1041652.777,...,0.0,1.0,0.0,0.0,2163,167,255,211,88,1
7,Balboa,Carminales,El Guadualito,6.607500e+09,30412314,6607500148,Puntual,COLOMBIA,794450.1679,1039293.955,...,1.0,1.0,0.0,0.0,2034,178,280,229,101,1
8,Balboa,Tambores,El Bosque,6.607501e+09,4551819,6607500516,Puntual,COLOMBIA,789741.6682,1033348.814,...,0.0,1.0,0.0,0.0,2072,168,263,216,94,1
9,Balboa,La Floresta,La Mejora,6.607501e+09,24512358,6607501187,Puntual,COLOMBIA,790532.0700,1036504.194,...,1.0,1.0,0.0,0.0,2131,166,256,211,89,1


In [73]:
for col in df.columns:
    print col
    

Municipio
Vereda
Finca
SICA
CEDULA 1
SICA 1
REPRESENTA
VARIEDAD
EPSG:3116_X
EPSG:3116_Y
EPSG:4326_Y
EPSG:4326_X
ASNM
LUMINOSID
Fecha Analysis
Malla 17-18
Malla 14-16
Malla <13
Negros y vinagres
Broca
Broca de punto
Veteado
Mordido
Inmaduro
Flojo
Sobresecado
Arrugado
Aplastado
Cristalizado
Reposado
Granizo
Conchas
Partido
Ambar
Defectos totales
Quaker (Nb Granos)
Fluorescentes (Nb Granos)
Pergamino
Almendra
AlmendraTotal
AlmendraSana
UV
Olor
Humedad (%)
Merma (%)
Aroma-Fragancia
Acidez
Cuerpo
Sabor
Sabor residual
Dulzor
Uniformidad
Balance
Taza limpia
Puntaje catador
Puntaje Total
Taza 1
Taza 2
Taza 3
Taza 4
Taza 5
prec1
prec2
prec3
prec4
prec5
prec6
prec7
prec8
prec9
prec10
tmin1
tmin2
tmin3
tmin4
tmin5
tmin6
tmin7
tmin8
tmin9
tmin10
tmax1
tmax2
tmax3
tmax4
tmax5
tmax6
tmax7
tmax8
tmax9
tmax10
tmean1
tmean2
tmean3
tmean4
tmean5
tmean6
tmean7
tmean8
tmean9
tmean10
dtr1
dtr2
dtr3
dtr4
dtr5
dtr6
dtr7
dtr8
dtr9
dtr10
year
PrecTotalAvg
TminTotalAvg
TmaxTotalAvg
TmeanTotalAvg
DtrTotalAvg
Soi

In [74]:



cols_ = ['SICA 1','EPSG:3116_X','EPSG:3116_Y','EPSG:4326_X','EPSG:4326_Y','Puntaje Total']
df_coordinates_pts = df[cols_]

df_coordinates_pts.to_csv("./data/Coordinates_Points_Risaralda_utf-8.csv", sep=',', encoding='utf-8')
df_coordinates_pts.to_csv("./data/Coordinates_Points_Risaralda_iso-8859-1.csv", sep=',', encoding='iso-8859-1')

In [75]:
#df_coordinates_pts

# Création d'un set contenant la variété

In [88]:
cols = ['SICA 1','VARIEDAD','ASNM','PrecTotalAvg','TminTotalAvg','TmaxTotalAvg','TmeanTotalAvg','DtrTotalAvg',
        "LUMINOSID", "pH_avg","org_avg","year","Defectos totales","Puntaje Total"]

df_compact_1 = df[cols].copy(deep=True)

df_compact_1['category'] = 0

for index, row in df_compact_1.iterrows():
    df_compact_1.set_value(index, 'category', classCoffee(row['Puntaje Total']))

df_compact_1.rename(columns={'Defectos totales': 'DefectosTotales'}, inplace=True)
df_compact_1.rename(columns={'SICA 1': 'SICA'}, inplace=True)
df_compact_1.rename(columns={'LUMINOSID': 'Luminosidad'}, inplace=True)
df_compact_1.rename(columns={'Defectos totales': 'DefectosTotales'}, inplace=True)
df_compact_1.rename(columns={'Puntaje Total': 'PuntajeTotal'}, inplace=True)

df_compact_1.to_csv("./data/df_compact_1_utf-8.csv", sep=',', encoding='utf-8')
df_compact_1.to_csv("./data/df_compact_1_iso-8859-1.csv", sep=',', encoding='iso-8859-1')




Variété
VARIEDAD       
CASTILLO  count     185.000000
          mean     1619.178378
          std       174.143106
          min      1047.000000
          25%      1536.000000
          50%      1637.000000
          75%      1744.000000
          max      2001.000000
CATURRA   count     335.000000
          mean     1641.564179
          std       153.884477
          min       935.000000
          25%      1552.500000
          50%      1644.000000
          75%      1750.000000
          max      1993.000000
COLOMBIA  count     217.000000
          mean     1544.004608
          std       149.512464
          min      1178.000000
          25%      1452.000000
          50%      1533.000000
          75%      1637.000000
          max      1957.000000
T?PICA    count       9.000000
          mean     1580.555556
          std       162.340777
          min      1344.000000
          25%      1500.000000
          50%      1596.000000
          75%      1639.000000
          max  

# Création d 'un set purement numérique

Attention certain champs comme "quarker" ou "fluorescentes" ont été supprimé étant donné qu'ils ne sont pas complet pour toutes les entrées.

Ci-dessous les colonnes que l'on souhaite voir dans DataRisaralda_v2Numeric_utf-8.csv 

In [ ]:
# Inputs: SICA, climat, soil
# Output: Physical caracteristics, degustation

cols = ['SICA 1',
        'prec1','prec2','prec3','prec4','prec5','prec6','prec7','prec8','prec9','prec10',
        'tmin1','tmin2','tmin3','tmin4','tmin5','tmin6','tmin7','tmin8','tmin9','tmin10',
        'tmax1','tmax2','tmax3','tmax4','tmax5','tmax6','tmax7','tmax8','tmax9','tmax10',
        'tmean1','tmean2','tmean3','tmean4','tmean5','tmean6','tmean7','tmean8','tmean9','tmean10',
        'dtr1','dtr2','dtr3','dtr4','dtr5','dtr6','dtr7','dtr8','dtr9','dtr10',
        'PrecTotalAvg','TminTotalAvg','TmaxTotalAvg','TmeanTotalAvg','DtrTotalAvg',
        'PrecTotal','TminTotal','TmaxTotal','TmeanTotal','DtrTotal',
        'year','ASNM', 'LUMINOSID',
        "pH_avg","org_avg",
        "franco_L1","arcilloso_L1","limoso_L1","arenoso_L1","cascajoso_L1",
        "franco_L2","arcilloso_L2","limoso_L2","arenoso_L2","cascajoso_L2",
        "franco_L3","arcilloso_L3","limoso_L3","arenoso_L3","cascajoso_L3",
        'Malla 17-18','Malla 14-16','Malla <13','Negros y vinagres','Broca','Broca de punto','Veteado','Mordido',
        'Inmaduro','Flojo','Sobresecado','Arrugado','Aplastado','Cristalizado','Reposado','Granizo','Conchas',
        'Partido','Ambar','Defectos totales','Humedad (%)','Merma (%)','Aroma-Fragancia',
        'Acidez','Cuerpo','Sabor','Sabor residual','Dulzor','Uniformidad','Balance','Taza limpia','Puntaje catador','Puntaje Total']

In [ ]:
df_num = df[cols]

In [ ]:
type(df_num['pH_avg'][2])

In [ ]:
df_num.to_csv("./data/DataRisaralda_v2Numeric_utf-8.csv", sep=',', encoding='utf-8')
df_num.to_csv("./data/DataRisaralda_v2Numeric_iso-8859-1.csv", sep=',', encoding='iso-8859-1')

# Dataset numérique complet
    sans données manquantes au niveau des colonnes !
    Les colonnes supprimées sont indiquées ci-dessous

In [ ]:
print df_num.shape
colsdfnum = df_num.columns

In [ ]:
df_num_complete = df_num.dropna(axis=1, how='any')
print df_num_complete.shape
colsdfnumcomplete = df_num_complete.columns

In [ ]:
deleted_cols = list(set(colsdfnum) - set(colsdfnumcomplete))
print("Colonnes supprimées: {}".format(deleted_cols))

# Renomage des colonnes

Attention à vérifier dans les scripts précédents

In [ ]:
df_num_complete.rename(columns={'Negros y vinagres': 'NegrosYVinagres'}, inplace=True)
df_num_complete.rename(columns={'Broca de punto': 'BrocaDePunto'}, inplace=True)
df_num_complete.rename(columns={'Defectos totales': 'DefectosTotales'}, inplace=True)
df_num_complete.rename(columns={'Humedad (%)': 'Humedad%'}, inplace=True)
df_num_complete.rename(columns={'Merma (%)': 'Merma%'}, inplace=True)
df_num_complete.rename(columns={'Sabor residual': 'SaborResidual'}, inplace=True)
df_num_complete.rename(columns={'Taza limpia': 'TazaLimpia'}, inplace=True)
df_num_complete.rename(columns={'Puntaje catador': 'PuntajeCatador'}, inplace=True)
df_num_complete.rename(columns={'Puntaje Total': 'PuntajeTotal'}, inplace=True)
df_num_complete.rename(columns={'SICA 1': 'SICA'}, inplace=True)


In [ ]:
df_num_complete.to_csv("./data/DataRisaralda_v2Numeric_Complete_utf-8.csv", sep=',', encoding='utf-8')
df_num_complete.to_csv("./data/DataRisaralda_v2Numeric_Complete_iso-8859-1.csv", sep=',', encoding='iso-8859-1')

In [ ]:
print(df_num_complete.shape)

# Insertion de la classe du café (sous forme numérique)

|Total Score|       Quality Classification||
|---|---|---|
|90-100	|          Outstanding|	             Specialty|
|85-89.99	|      Excellent||
|80-84.99|	      Very Good||
|<80.0	|          Below Specialty Quality|	 Not Specialty|


http://www.scaa.org/?page=resources&d=cupping-protocols

In [ ]:
df_num_complete_class = df_num_complete.copy(deep=True)
df_num_complete_class['category'] = 0

for index, row in df_num_complete_class.iterrows():
    df_num_complete_class.set_value(index, 'category', classCoffee(row['PuntajeTotal']))

In [ ]:
print df_num_complete_class.columns
for e in df_num_complete_class['category']:
    print e

## Utilisé spécialement dans R 
Remplacement des noms avec espaces -> trim

In [ ]:
df_num_complete_class.to_csv("./data/DataRisaralda_v2Numeric_Complete_class_utf-8.csv", sep=',', encoding='utf-8')
df_num_complete_class.to_csv("./data/DataRisaralda_v2Numeric_Complete_class_iso-8859-1.csv", sep=',', encoding='iso-8859-1')

In [ ]:
df_num_complete_class

# Création d'un set GPS - Class

In [ ]:


df_coords_class = df_coordinates_pts.copy(deep=True)

df_coords_class['category'] = 0

for index, row in df_coords_class.iterrows():
    df_coords_class.set_value(index, 'category', classCoffee(row['Puntaje Total']))
del df_coords_class['Puntaje Total']

In [ ]:
df_coords_class.to_csv("./data/Coordinates_Class_Risaralda_utf-8.csv", sep=',', encoding='utf-8')
df_coords_class.to_csv("./data/Coordinates_Class_Risaralda_iso-8859-1.csv", sep=',', encoding='iso-8859-1')
df_coords_class